In [1]:
import sqlite3
import os
import pandas as pd
import pdfkit    
from pathlib import Path
import shutil
import csv
from datetime import datetime
from db.database import conectar, excluirTabela, tabelaLivros, adicionarLivro, atualizarPreco, removerLivro, buscarLivroPorAutor, exibirLivros, obter_dados, obter_dados_livros

#Diretórios          
BASE_DIR = Path.cwd()
BACKUP_DIR = BASE_DIR / 'backups'
DATA_DIR = BASE_DIR / 'data'
EXPORT_DIR = BASE_DIR / 'exports'
DB_FILE = DATA_DIR / 'livraria.db'

#Criar as pastas
def setup_directories():
    os.makedirs(BACKUP_DIR, exist_ok=True)
    os.makedirs(DATA_DIR, exist_ok=True)
    os.makedirs(EXPORT_DIR, exist_ok=True)

#Função para conectar o banco
def main():
    conn = conectar('data/livraria.db')
#  inserir_dados(conn, dados)

# Inicializando a estrutura do projeto
setup_directories()
tabelaLivros(DB_FILE)
#excluirTabela(DB_FILE)

In [304]:
def exportarCsv(dados, colunas, nome_arquivo):
    with open(nome_arquivo, mode='w', newline='', encoding='utf-8') as arquivo_csv:
        writer = csv.writer(arquivo_csv)
        
        # Escrever o cabeçalho
        writer.writerow(colunas)

        # Escrever os dados
        for linha in dados:
            writer.writerow(linha)

In [1]:
def importar_csv(caminho_arquivo, sep=',', encoding='utf-8'):
    
    try:
        # Ler o arquivo CSV
        df = pd.read_csv(caminho_arquivo, sep=sep, encoding=encoding)
        
        # Exibir informações gerais sobre os dados
        print(df.info())
        
        # Mostrar as primeiras 5 linhas do DataFrame
        print(df.head())
        
        return df
    except FileNotFoundError:
        print("Arquivo não encontrado. Verifique o caminho.")
        
    except pd.errors.EmptyDataError:
        print("O arquivo está vazio.")
        
    except pd.errors.ParserError:
        print("Erro ao analisar o arquivo.")

In [4]:
def realizarBackupDB():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    nome_backup = f"backup_{timestamp}"
    path_backup = os.path.join(BACKUP_DIR, nome_backup)

    # Copiar o diretório do projeto para o diretório de backup
    shutil.copy(DATA_DIR, path_backup)

    print(f"Backup do banco de dados criado em: {BACKUP_DIR}")

In [307]:
def gerar_relatorio_html(df, EXPORT_DIR):
    
    html = df.to_html(index=False)
    with open(EXPORT_DIR, 'w') as f:
        f.write(html)
    print(f"Relatório HTML salvo em {EXPORT_DIR}")

In [308]:
def gerar_relatorio_pdf(df, EXPORT_DIR):
    html = df.to_html(index=False)
    pdfkit.from_string(html, EXPORT_DIR)
    print(f"Relatório PDF salvo em {EXPORT_DIR}")

In [309]:
def adicionar_livro(DB_FILE):
    titulo = input("Título do livro: ")
    autor = input("Autor do livro: ")
    
    #Validação de ano
    while True:
        try:
            ano_publicacao = int(input("Ano de Publicação: "))
            if 1000 <= ano_publicacao <= datetime.now().year:
                break
            else:
                print(f"Por favor, insira um ano válido entre 1000 e {datetime.now().year}.")
        except ValueError:
            print("Ano de publicação deve ser um número inteiro.")

    #Validação do preço
    while True:
        try:
            preco = float(input("Preço do livro (R$): "))
            if preco >= 0:
                break
            else:
                print("O preço deve ser um valor positivo.")
        except ValueError:
            print("Por favor, insira um valor numérico válido para o preço.")

    conn = conectar(db_path)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO livros (titulo, autor, ano_publicacao, preco) VALUES (?, ?, ?, ?)", 
                   (titulo, autor, ano_publicacao, preco))
    conn.commit()
    conn.close()
    print(f'Livro "{titulo}" adicionado com sucesso!')
    #adicionar função e fazer backup automático


In [310]:
def exibir_livros(DB_FILE):
    conn = conectar(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM livros")
    livros = cursor.fetchall()
    conn.close()
    
    if livros:
        print("\n=== Lista de Livros Cadastrados ===")
        for livro in livros:
            print(f'ID: {livro[0]}, Título: {livro[1]}, Ano: {livro[3]}, Preço: R${float(livro[4]):.2f}')
    else:
        print("Nenhum livro cadastrado.")


In [311]:
def atualizar_preco(DB_FILE):
    livro_id = input("ID do livro a ser atualizado: ")
    
    #Validação do novo preço
    while True:
        try:
            novo_preco = float(input("Novo preço do livro (R$): "))
            if novo_preco >= 0:
                break
            else:
                print("O preço deve ser um valor positivo.")
        except ValueError:
            print("Por favor, insira um valor numérico válido.")
    
    conn = conectar(db_path)
    cursor = conn.cursor()
    cursor.execute("UPDATE livros SET preco = ? WHERE id = ?", (novo_preco, livro_id))
    conn.commit()
    conn.close()
    print(f'Preço do livro com ID {livro_id} atualizado para R${novo_preco:.2f}')
    #adicionar função e fazer backup automático

In [312]:
def remover_livro(DB_FILE):
    livro_id = input("ID do livro a ser removido: ")
    
    conn = conectar(db_path)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM livros WHERE id = ?", (livro_id,))
    conn.commit()
    conn.close()
    print(f'Livro com ID {livro_id} removido com sucesso!')
    #adicionar função e fazer backup automático


In [313]:
def buscar_livros_por_autor(DB_FILE):
    autor = input("Digite o nome do autor: ")

    cursor = conn.cursor()
    cursor.execute("SELECT * FROM livros WHERE autor = ?", (autor,))
    livros = cursor.fetchall()
    conn.close()

    if livros:
        print(f"\n=== Livros de {autor} ===")
        for livro in livros:
                print(f'ID: {livro[0]}, Título: {livro[1]}, Ano: {livro[3]}, Preço: R${float(livro[4]):.2f}')
        else:
            print("Nenhum livro cadastrado.")




In [3]:
def exibir_menu_jupyter():
    opcao = None
    
    while opcao != '11':
        print("\n=== Sistema de Gerenciamento de Livraria ===")
        print("1. Adicionar novo livro")
        print("2. Exibir todos os livros")
        print("3. Atualizar preço de um livro")
        print("4. Remover um livro")
        print("5. Buscar livros por autor")
        print("6. Exportar dados para CSV")
        print("7. Exportar dados para HTML")
        print("8. Exportar dados para PDF")
        print("9. Importar dados de CSV")
        print("10. Fazer backup do banco de dados")
        print("11. Sair")
        
        opcao = input("Escolha uma opção: ")

        if opcao == '1':
            adicionarLivro(DB_FILE)
        
        elif opcao == '2':
            exibirLivros(DB_FILE)
        
        elif opcao == '3':
            atualizarPreco(DB_FILE)
        
        elif opcao == '4':
            removerLivro(DB_FILE)
        
        elif opcao == '5':
            buscarLivroPorAutor(DB_FILE)
        
        elif opcao == '6':
            dados, colunas = obter_dados(DB_FILE)
            exportarCsv(dados, colunas, './exports/livros_exportados.csv')
            
        elif opcao == '7':
            df_livros = obter_dados_livros(DB_FILE)
            gerar_relatorio_html(df_livros, './exports/livros_exportados.html')
            
        elif opcao == '8':
            df_livros = obter_dados_livros(DB_FILE)
            gerar_relatorio_pdf(df_livros, './exports/livros_exportados.pdf')
        
        elif opcao == '9':
            print("Por favor, anexe o arquivo .csv na pasta import com o nome livro_importado.csv")
            df = importar_csv('./import/livro_importado.csv') 
        
        elif opcao == '10':
            realizarBackupDB()
        
        elif opcao == '11':
            print("Saindo...")
        else:
            print("Opção inválida. Tente novamente.")
        
        if opcao != '11':
            continuar = input("Deseja realizar outra operação? (s/n): ")
            if continuar.lower() != 's':
                print("Saindo...")
                break

exibir_menu_jupyter()


=== Sistema de Gerenciamento de Livraria ===
1. Adicionar novo livro
2. Exibir todos os livros
3. Atualizar preço de um livro
4. Remover um livro
5. Buscar livros por autor
6. Exportar dados para CSV
7. Exportar dados para HTML
8. Exportar dados para PDF
9. Importar dados de CSV
10. Fazer backup do banco de dados
11. Sair


Nenhum livro cadastrado.
Saindo...
